This project requires Python 3.7 or above:

In [1]:
import sys

assert sys.version_info >= (3, 7)

It also requires Scikit-Learn >/ 1.0.1:

In [2]:
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

# Get the data

# Download the Data

In [3]:
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns

houseprice = pd.read_csv("houseprice.csv")

## Cleaning the data

Create a copy of the data as to not touch the raw data.

In [4]:
df = houseprice.copy()

In [5]:
def convert_lot_size(value):
    if pd.isna(value):
        return np.nan
    s = str(value).lower()
    if "acre" in s:
        num = float(s.replace("acres", "").replace("acre", "").strip())
        return num * 43560 #1 acre = 43560 sqft
    elif "sqft" in s:
        num = float(s.replace("sqft", "").strip())
        return num
    else:
        try:
            return float(s)
        except:
            return np.nan
        
if "Lot Size" in df.columns:
    df["Lot Size"] = df["Lot Size"].apply(convert_lot_size)

# Drop Duplicates

df.drop_duplicates(inplace=True)

#Remove rows with zero or negative Price or Area
if "Price" in df.columns:
    df = df[df["Price"] > 0]
if "Square Feet" in df.columns:
    df = df[df["Square Feet"] > 0]

# Handling outliers by capping extreme values
numeric_cols = df.select_dtypes(include=np.number).columns
for col in numeric_cols:
    upper = df[col].quantile(0.99)
    df[col] = np.where(df[col] > upper, upper, df[col])

# Making Zipcode categorical and not just numeric
if "Zipcode" in df.columns:
    df["Zipcode"] = df["Zipcode"].astype(str)

# Checking for missing values.
print("Missing values per column:")
print(df.isna().sum())

df.head()

ValueError: could not convert string to float: '7,895'